# Ettoday 網路爬蟲實作練習


* 能夠利用 Request + BeatifulSour 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例：

1. 取出今天所有的新聞
2. 取出現在時間兩小時內的新聞
3. 根據範例，取出三天前下午三點到五點的新聞

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup

import time
import re
import pandas as pd

## 1. 取出今天所有的新聞

In [2]:
chromedriver = 'C:/ProgramData/Anaconda3/Lib/site-packages/chromedriver.exe'
browser = webdriver.Chrome(executable_path = chromedriver)
browser.get("https://www.ettoday.net/news/news-list.htm")

'''
for i in range(50):
    time.sleep(2)
    browser.execute_script("window.scrollTo(0, 10000);")
'''

#browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

SCROLL_PAUSE_TIME = 2

# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")


while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    
    html_source = browser.page_source
    soup = BeautifulSoup(html_source, "html5lib")
    max_len = len(soup.find(class_='part_list_2').find_all('h3')) - 1
    max_date = soup.find(class_="part_list_2").find_all('h3')[max_len].find(class_="date").text
    
    if re.match('2020/10/10', max_date) != None:
        break
    elif new_height == last_height:
        break
    else: last_height = new_height

html_source = browser.page_source

browser.close()

In [58]:
soup = BeautifulSoup(html_source, "html5lib")

'''
for d in soup.find(class_="part_list_2").find_all('h3'):
    print(d.find(class_="date").text, d.find_all('a')[-1].text)
'''

#df_today = pd.DataFrame(columns = ['date','title'])
df_today = []


for d in soup.find(class_="part_list_2").find_all('h3'):
    sub_time = d.find(class_="date").text 
    sub_date = d.find_all('a')[-1].text
    #sub = sub_time, sub_date
    sub = [sub_time, sub_date]
    #sub = DataFrame(sub)
    #sub = list(sub_time, sub_date)
    #print(sub)
    df_today.append(sub)
    
df_today = pd.DataFrame(df_today, columns = ['date','title'])
#df_today = df_today.drop(index = len(df_today) - 1) #刪掉最後一條  為昨日新聞

df_today = df_today[df_today.date.str.contains('2020/10/11.*', regex= True).astype('uint8') != 0]

In [59]:
print(df_today)

                 date                                    title
0    2020/10/11 10:33          絕育觀念陌生！餓死小貓減緩數量　台灣之心進駐馬祖結紮81浪犬貓
1    2020/10/11 10:32         火鍋店調戲正妹店員　2男「10秒被打趴」...超猛真實身分曝光！
2    2020/10/11 10:30                   中菲外長會談　磋商南海行為準則、開展疫苗合作
3    2020/10/11 10:30            來嘉義吃喝玩樂！故宮南院浪漫二日遊　還有露天泳池、空靈茶園
4    2020/10/11 10:29        「這4生肖」2020年倒吃甘蔗：一路旺到年底！　冠軍運勢紅到你會怕
5    2020/10/11 10:27          手指餅乾、搞怪甜甜圈、惡魔之眼好有Fu　20元起過萬聖節無負擔
6    2020/10/11 10:26          法網／斯威雅蒂28年來最年輕冠軍　波蘭史上第一人排名衝上第17
7    2020/10/11 10:24                倒數2分鐘內2次誤判！　G5裁判報告「對湖人不利」
8    2020/10/11 10:21                   日職／金鶯隊友再聚首　日媒：和田毅警戒陳偉殷
9    2020/10/11 10:19          接駁車上看完國慶煙火！他怒轟：等車2小時+塞車4小時　網全戰翻
10   2020/10/11 10:17           2019平均所得逾66萬元　45至54歲者賺最多、逾85萬元
11   2020/10/11 10:16               蓬佩奧：中國6萬重兵現身印北邊界！　軍事對峙未見降溫
12   2020/10/11 10:16       一倒就噴3萬！他Gogoro被撞倒「挨轟敲詐維修費」　網秒懂：變孝子
13   2020/10/11 10:12             大媽斥「台語很土，請講國語」　她當場回擊！曝兒時心痛記憶
14   2020/10/11 10:11       「神秘兩小時」喝自來水　湖北20多人上吐下瀉...有人

## 2. 取出現在時間兩小時內的新聞

In [34]:
type(df_today.date.str.match('2020/10/10.*'))

pandas.core.series.Series

In [60]:
df_today_09 = df_today[df_today.date.str.contains('2020/10/11 09:.*', regex= True).astype('uint8') != 0]
df_today_10 = df_today[df_today.date.str.contains('2020/10/11 10:.*', regex= True).astype('uint8') != 0]

df_today_2hr = df_today_09.append(df_today_10)


In [61]:
print(df_today_2hr)

                date                                title
25  2020/10/11 09:58             「岳跳岳高」岳東華演超狂美技　為米蘭達抓關鍵出局
26  2020/10/11 09:56             道奇國冠戰先發輪值沒變　G1布勒先發、克蕭投G2
27  2020/10/11 09:49            台中總站夜市大跳電　台電：業者用「材料堆積場」申請
28  2020/10/11 09:47         國慶收假日海象不佳　基隆-馬祖、布袋-馬公等19航次停航
29  2020/10/11 09:46        金雨樹飄落秋日浪漫！新北2處欒樹秘境「騎鐵馬道穿梭」超療癒
30  2020/10/11 09:45         快訊／國慶連假共機全勤擾台！　20分鐘內來兩次連6天報到
31  2020/10/11 09:44       陸使館氣炸施壓遵守一中！印度記者遭約「喝茶」笑：北京早被孤立
32  2020/10/11 09:44    瘋狂網購18年「塞爆2屋」！家人嚇壞…找拍賣發現竟是1.5億元收藏
33  2020/10/11 09:39         老鄉才知道！台南隱藏版市集美食　豬肝小卷舖滿滿只要60元
34  2020/10/11 09:36       看書放空賞巨龜！高雄六星級書店咖啡　還能吃到法式主廚銷魂燉飯
35  2020/10/11 09:31         共機頻擾家長擔心小孩被送上戰場　沈富雄斷言：兩岸不會開打
36  2020/10/11 09:31    狠父逼吞藥下重拳！英2歲女嬰臉腫脹險死　10年後尋根「真相更黑暗」
37  2020/10/11 09:24             高屏地區紅豆苗期　高雄區農改場籲農友留意根腐病　
38  2020/10/11 09:21      孕婦中標害胎兒感染！水痘「湯汁流出」6大殺傷力曝...慘變皮蛇
39  2020/10/11 09:19      1家4口困電梯1.5小時！小妹妹：謝謝消防叔叔沒吃晚餐來救我們
40  2020/10/11 09:15       倒入飲料就出現字！日本爆紅「透明玻璃杯」　破40萬讚跪求量產
41  2020/10/11

## 3. 根據範例，取出三天前下午三點到五點的新聞

In [81]:
chromedriver = 'C:/ProgramData/Anaconda3/Lib/site-packages/chromedriver.exe'
browser = webdriver.Chrome(executable_path = chromedriver)
browser.get("https://www.ettoday.net/news/news-list-2020-10-8-0.htm")

'''
for i in range(50):
    time.sleep(2)
    browser.execute_script("window.scrollTo(0, 10000);")
'''

#browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

SCROLL_PAUSE_TIME = 10

# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")


while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    
    html_source = browser.page_source
    soup = BeautifulSoup(html_source, "html5lib")
    max_len = len(soup.find(class_='part_list_2').find_all('h3')) - 1
    max_date = soup.find(class_="part_list_2").find_all('h3')[max_len].find(class_="date").text
    
    if re.match('2020/10/07', max_date) != None:
        break
    elif new_height == last_height:
        break
    else: last_height = new_height

html_source = browser.page_source

browser.close()

In [85]:
soup = BeautifulSoup(html_source, "html5lib")

'''
for d in soup.find(class_="part_list_2").find_all('h3'):
    print(d.find(class_="date").text, d.find_all('a')[-1].text)
'''

#df_today = pd.DataFrame(columns = ['date','title'])
df_d3 = []


for d in soup.find(class_="part_list_2").find_all('h3'):
    sub_time = d.find(class_="date").text 
    sub_date = d.find_all('a')[-1].text
    #sub = sub_time, sub_date
    sub = [sub_time, sub_date]
    #sub = DataFrame(sub)
    #sub = list(sub_time, sub_date)
    #print(sub)
    df_d3.append(sub)
    
df_d3 = pd.DataFrame(df_d3, columns = ['date','title'])
#df_today = df_today.drop(index = len(df_today) - 1) #刪掉最後一條  為昨日新聞

df_d3 = df_d3[df_d3.date.str.contains('2020/10/08.*', regex= True).astype('uint8') != 0]

In [86]:
print(df_d3)

                 date                                title
0    2020/10/08 23:59     快訊／33層樓「燒成火柱」！蔚山公寓深夜大火　137戶上百人驚逃
1    2020/10/08 23:53       佟麗婭「與帥哥牽手親密照」曝光！　親上火線還原現場：別編劇本
2    2020/10/08 23:48           台灣好行限定套票即將開賣　住大溪老城區輕鬆漫步東眼山
3    2020/10/08 23:46              教士G3非贏不可　推21歲左投莫雷洪先發抗道奇
4    2020/10/08 23:41     饒河夜市「地瓜父子」真相曝光！老闆娘784字吐心聲　柯P現身回應
5    2020/10/08 23:29             賈乃亮度假被偶遇要合照　一聽「最愛甜馨」臉僵了！
6    2020/10/08 23:22            快訊／白色休旅車二殯前衝撞工程車　前方保險桿脫落！
7    2020/10/08 23:20        陳廷寵批「國軍戰力0」　顏若芳：您身為前陸軍總司令沒責任？
8    2020/10/08 23:19       快訊／新莊17歲少年遭流彈打中下體　送醫途中「救護車出車禍」
9    2020/10/08 23:19   GIVENCHY新總監首個聯名「黑魂果汁」出爐　網抗議：我要的是衣服
10   2020/10/08 23:17       明年4月要退伍！金門車長林楷強殉職　媽媽曾盼「再半年就團圓」
11   2020/10/08 23:13      韓選美皇后曬「胸間挖空」神級泳衣照！　火辣指數爆表…網直噴鼻血
12   2020/10/08 23:09            等很久了！　機捷A21站地區區段徵收內政部審議通過
13   2020/10/08 23:07                 中職／打不好萬昭清　丘總：不適應陌生投手
14   2020/10/08 23:00        快訊／新莊夜市旁深夜槍響！路人下體遭流彈打中　2人送醫急救
15   2020/10/08 22:56                中職／雷艾斯右手肘有問題　丘總：應該會

In [91]:
df_d3_15 = df_d3[df_d3.date.str.contains('2020/10/08 15:.*', regex= True).astype('uint8') != 0]
df_d3_16 = df_d3[df_d3.date.str.contains('2020/10/08 16:.*', regex= True).astype('uint8') != 0]
df_d3_17 = df_d3[df_d3.date.str.contains('2020/10/08 17:.*', regex= True).astype('uint8') != 0]

df_d3_1517 = df_d3_15.append(df_d3_16)
df_d3_1517 = df_d3_1517.append(df_d3_17)

In [92]:
print(df_d3_1517)

                 date                                            title
306  2020/10/08 15:59                《看臉時代》花美作者老婆也超正！「差點成為MAMAMOO」背景曝光
307  2020/10/08 15:59                     獨行俠老闆稱詹皇與9年前「差很大」　球商變很高成籃球大師
308  2020/10/08 15:57                         快訊／太保市水牛厝公園民眾溺水　無生命徵象搶救中
309  2020/10/08 15:56              《玩很大》新制服曝光！　左邊心臟刻「小鬼專屬印記」…網鼻酸：他永遠都在
310  2020/10/08 15:55                        考選部調整考試方式！提高口試比重　引進優質專技人才
311  2020/10/08 15:54                         理我！奴才進食中小貴賓「爬空盤」　趴滿萌刷存在感
312  2020/10/08 15:53                            中職／運彩風險控管不開盤　象猿大戰已有前例
313  2020/10/08 15:53                         智冠科技9月營收年增19%　受惠展會活動業務回溫
314  2020/10/08 15:53          楊丞琳露腰秀包8連拍！LOEWE Puzzle讓人心動的5個理由　焦糖色好秋天
315  2020/10/08 15:52                        華南銀推智能理財　AI判斷、解決痛點　降低投資失準
316  2020/10/08 15:51                      【黃寶慧】慧眼看天下／美選版兩顆子彈？川普染疫民調逆襲
317  2020/10/08 15:49                    被砍重傷昏迷要簽名才能告！家屬怒嗆檢警：以後最好打成植物人
318  2020/10/08 15:48                  手滑誤刪「電話」App！網友3招找回來　曝移除真相：只是隱藏了
319  2